<a href="https://colab.research.google.com/github/gianellannie/telecom-x-parte2/blob/main/TelecomX_LATAM_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Preparación de los Datos**

# **2. Correlación y Selección de Variables**

# **3. Modelado Predictivo**

# **4.Interpretación y Conclusiones**